In [1]:
import time

In [1]:
# Because I want to play with selenium a bit
# grabbing the webdriver from https://github.com/mozilla/geckodriver/releases
# I put that in my user bin: /Users/emily/bin ... nope, needs to be in /usr/local/bin.



# And I tried installing chromedriver via brew, but that didn't seem right. 
# So, I downloaded & that's going in /usr/local/bin as well

## Some nice references:
    http://www.marinamele.com/selenium-tutorial-web-scraping-with-selenium-and-python

In [2]:
9100*20/(60*60*24)

2.1064814814814814

# Examining SkinDeep data.
### Let's start exploring the structure of the webpage.
* [Starting access point](https://www.ewg.org/skindeep/search?search=%2A&search_type=ingredients)
* [The robots.txt file](https://www.ewg.org/robots.txt)
    The crawl delay is set to 10.

In that page, there are (as of Nov 2019) 9069 ingredients listed in the database.
So, it would take a little over a day of straight scraping at that crawl delay.
That's fine, I'll just work slowly and make sure to write out the data often.

#### Breakdown of ingredient results:
section class="product-listings" -> div class="product-tile"
    -> a href to the page for that ingredient.

#### Navigation of ingredients page
div class="content" -> section id="chemical":

    To get name:
        h2 class="chemical-name text-block"
    To get score: will need to parse name of image:
        div class="chemical-score float-r"
            <p>"Score:"</p>
                <img>  with class "squircle"
    To get amount of Data available:
        div class="chemical-score float-r"
            <p>"Data: "</p>
                <span> 
    To get quick details about the ingredient:
        section class="see-all-chemical"
            a class="chemical-concerns"
                (inside a <li>, hidden depends on what's clicked:)
                p class="chemical-info chemical-concerns-text"
                p class="chemical-info chemical-functions-text hidden"
                p class="chemical-info chemical-about-text hidden"
                p class="chemical-info chemical-synonyms-text hidden"
         
    
 
We could either (1) pull off all the links from the search listings, then navigate to all those directly, or (2) go through all the search listings and navigate to links as we go.

Not sure which is better yet.
As long as I write out all the bits I'm interested in, I think I'll be okay.


In [ ]:
# Looking at one of the sample EWG file
TODO COPY OVER THOSE THIGNS FROM THE OTHER NOTEBOOK

In [1]:
# Okay, lets dig into selenium a bit more

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException, NoSuchElementException



In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get("https://www.ewg.org/skindeep/search?search=*&search_type=ingredients&per_page=36")

In [ ]:
# Oh look, there's going to be a pop up about donating, that we may have to deal with.
# <body> -> div id="yeaFixed" -> a href id="yeaClose"

In [3]:
driver.find_element_by_id('yeaClose').click() # Click on this yeaClose element in order to get to the new 
# will want to wrap in a try/except  (ElementNotInteractableException)


In [39]:
# Get the elements with this class - returns a list.
# This will 
pt = driver.find_elements_by_class_name('product-tile')
[elem.text for elem in pt[0].find_elements_by_tag_name("a")]

a_tag = pt[0].find_elements_by_tag_name("a")[1]

In [40]:
a_tag.get_attribute('href')

'https://www.ewg.org/skindeep/ingredients/861672-HYPOXIS_HEMEROCALLIDEA_AFRICAN_WILD_POTATO_ROOT_EXTRACT'

In [4]:
np = driver.find_element_by_class_name('next_page')

In [7]:
driver.find_element_by_class_name('next_page').get_attribute('href')

'https://www.ewg.org/skindeep/search?page=2&per_page=36&search=%2A&search_type=ingredients'

In [ ]:
# Both "a" tags in product-tile link to the same place.
# We're going to use the second one so we can pull off this text and record it.

In [121]:
pt = driver.find_elements_by_class_name('product-tile')

# To iterate through all the product-tiles:
for i, elem in enumerate(pt):
    
    a_tag = elem.find_elements_by_tag_name("a")[1]
    a_tag.click()
    time.sleep(10)
    driver.back()


AttributeError: 'WebDriver' object has no attribute 'navigate'

In [123]:
driver.close()

In [ ]:
# Okay, so now on each page, let's figure out the parts we want to pull
# from the individual chemical page


In [124]:
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/skindeep/ingredients/702620-GLYCERIN")


In [142]:
chemical_score_elem = driver.find_element_by_xpath('//*[@id="chemical"]/div[2]/img')
image_link_text = chemical_score_elem.get_attribute('src')

In [143]:
image_link_text # success!

'https://www.ewg.org/skindeep/squircle/show.svg?score=2&score_min=1'

In [145]:
chemical_name = driver.find_element_by_xpath('//*[@id="chemical"]/h2')
chemical_name.text # looking good

'GLYCERIN'

In [82]:
chemical_info = driver.find_elements_by_class_name('chemical-info')

In [85]:
[elem.text for elem in chemical_info]

['',
 '',
 '',
 '1385RN 59, 1700 WHITE, 234DA, 500HD, 63B1 WHITE, A 200 (PIGMENT), A 330 (PIGMENT), A-FIL, A-FIL CREAM, A-FN 3, AEROLYST 7710, AEROSIL P 25, AEROSIL P 25S6, AEROSIL P 27, AEROSIL T 805, AI3-01334, AK 15 (PIGMENT), AMORPHOUS TITANIUM DIOXIDE, AMPERIT 780.0, AMT 100, AMT 600, ATLAS WHITE TITANIUM DIOXIDE, AUF 0015S, AUSTIOX R-CR 3, B 101 (PIGMENT), BAYER R-FD 1, BAYERTITAN A, BAYERTITAN AN 3, BAYERTITAN R-FD 1, BAYERTITAN R-FK 21, BAYERTITAN R-FK-D, BAYERTITAN R-KB 2, BAYERTITAN R-KB 3, BAYERTITAN R-KB 4, BAYERTITAN R-KB 5, BAYERTITAN R-KB 6, BAYERTITAN R-U 2, BAYERTITAN R-U-F, BAYERTITAN R-V-SE 20, BAYERTITAN T, BISTRATER L-NSC 200C, BLEND WHITE 9202, BR 29-7-2, BROOKITE, C 97 (OXIDE), C-WEISS 7, C.I. 77891, C.I. PIGMENT WHITE 6, CAB-O-TI, CALCOTONE WHITE T, CCRIS 590, CG-T, CI 77891, CI PIGMENT WHITE 6, CL 310, COSMETIC HYDROPHOBIC TIO2 9428, COSMETIC MICRO BLEND TIO2 9228, COSMETIC WHITE C47-5175, COSMETIC WHITE C47-9623, E 171, EINECS 236-675-5, FLAMENCO, HOMBITAN, HO

In [86]:
chemical_info[3].text

'1385RN 59, 1700 WHITE, 234DA, 500HD, 63B1 WHITE, A 200 (PIGMENT), A 330 (PIGMENT), A-FIL, A-FIL CREAM, A-FN 3, AEROLYST 7710, AEROSIL P 25, AEROSIL P 25S6, AEROSIL P 27, AEROSIL T 805, AI3-01334, AK 15 (PIGMENT), AMORPHOUS TITANIUM DIOXIDE, AMPERIT 780.0, AMT 100, AMT 600, ATLAS WHITE TITANIUM DIOXIDE, AUF 0015S, AUSTIOX R-CR 3, B 101 (PIGMENT), BAYER R-FD 1, BAYERTITAN A, BAYERTITAN AN 3, BAYERTITAN R-FD 1, BAYERTITAN R-FK 21, BAYERTITAN R-FK-D, BAYERTITAN R-KB 2, BAYERTITAN R-KB 3, BAYERTITAN R-KB 4, BAYERTITAN R-KB 5, BAYERTITAN R-KB 6, BAYERTITAN R-U 2, BAYERTITAN R-U-F, BAYERTITAN R-V-SE 20, BAYERTITAN T, BISTRATER L-NSC 200C, BLEND WHITE 9202, BR 29-7-2, BROOKITE, C 97 (OXIDE), C-WEISS 7, C.I. 77891, C.I. PIGMENT WHITE 6, CAB-O-TI, CALCOTONE WHITE T, CCRIS 590, CG-T, CI 77891, CI PIGMENT WHITE 6, CL 310, COSMETIC HYDROPHOBIC TIO2 9428, COSMETIC MICRO BLEND TIO2 9228, COSMETIC WHITE C47-5175, COSMETIC WHITE C47-9623, E 171, EINECS 236-675-5, FLAMENCO, HOMBITAN, HOMBITAN R 101D, H

In [84]:
driver.find_element_by_class_name('chemical-synonyms').click()

'Use restrictions (moderate)'

In [196]:
# I'll need to click on the appropriate links to pull up the data to read
# And then read using chemical_info = driver.find_elements_by_class_name('chemical-info')

In [ ]:
# And I forgot to look at how to advance pages...


In [6]:
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/skindeep/search?search=*&search_type=ingredients&per_page=36")


try:
    # If there's a pop-up, let's close it.
    driver.find_element_by_id('yeaClose').click()

except ElementNotInteractableException:
    pass
    


In [10]:
next = driver.find_element_by_class_name('next_page')

In [17]:
driver.find_element_by_class_name('next_page').is_displayed()

False

In [26]:
try:
    # If there's a pop-up, let's close it.
    driver.find_element_by_id('yeaClose').click()

except NoSuchElementException:
    pass

finally:
    driver.find_element_by_class_name('previous_page').click()

In [42]:
# driver.find_element_by_class_name('next_page').click() 
# yep, goes to next page.
# turns off when 

In [ ]:
driver.close()

In [ ]:
### Okay, let's clear up how we can parse out what we want from a product page


In [43]:
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/skindeep/ingredients/706561-TITANIUM_DIOXIDE")

In [58]:
name = driver.find_elements_by_class_name('chemical-score')

In [59]:
name

[<selenium.webdriver.remote.webelement.WebElement (session="98b4b35b65ff7c11ae84a3512dd86b50", element="4d9ee010-2d54-4c1d-ac60-556c39a6188c")>]

In [74]:
driver.find_element_by_class_name('squircle').get_attribute('src')

'https://www.ewg.org/skindeep/squircle/show.svg?score=3&score_min=1'

In [ ]:
score = driver.find_elements_by_class_name('')